In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_tuner as kt
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input, LeakyReLU
from keras.utils import np_utils
import datetime
from keras.applications.vgg16 import VGG16
from keras.models import Sequential

import os, warnings
from tensorflow.keras.utils import image_dataset_from_directory

# Project 2 - Veggie Classification

For this assignment you'll need to classify some images of vegetables. 

## Parts

Please do two separate classifications:
<ol>
<li> First, create a model from scratch. 
<li> Use transfer learning to use a pretrained model of your choice, adapted to this data. 
</ol>

There won't be an explicit evaluation of accuracy, but you should take some steps to make each model as accurate as you reasonably can, any tuning option is fair game. Along with that, please structure it into a notebook that is well structured and clear that explains what you did and found. Think about:
<ul>
<li> Sections and headings. 
<li> A description of the approach taken (e.g. what did you do to determine size, tune, evaluate, etc...)
<li> Visualization of some important things such as a confusion matrix and maybe some images. 
<li> Results, mainly focused on the scoring of the test data. 
</ul>

The descriptions and explainations should highlight the choices you made and why you made them. Figure up to about a page or so worth of text total, explain what happened but don't write an essay. 

## Deliverables

Please sumbmit a link to your github, where everyhting is fully run with all the outputs showing on the page. As well, in the notebook please add some kind of switch controlled by a variable that will control if the notebook runs to train the model or to load the model in from weights - so I can download it and click run all, it will load the saved weights, and predict.

### Dataset

The code in the start of this notebook will download and unzip the dataset, and there is also a simple example of creating datasets. You can change the dataset bit to use a different approach if you'd like. The data is already split into train, validation, and test sets. Please treat the separate test set as the final test set, and don't use it for any training or validation. Each folder name is its own label.

### Evaluation

Marking will be based on the following:
<ul>
<li> Models are cretaed, tuned, and effective at classifying the data: 40%
<li> Descriptions and explanations of the approach taken: 20%
<li> Code is well structured and clear: 20%
</ul>

Overall the marking is pretty simple and direct, walk through the process of predicting the veggies, explain what you did, and show the results. If you do that, it'll get a good mark.

### Tips

Some hints that may be helpful to keep in mind:
<ul>
<li> The data is pretty large, so you'll want to use datasets rather than load everything into memory. The Keras docs have a few examples of different ways to load image data, our examples showed image generators and the image from directory datasets.  
<li> Be careful of batch size, you may hit the colab limits. 
<li> You'll want to use checkpoints so you can let it train and pick up where you left off.
<li> When developing, using a smaller dataset sample is a good idea. These weights could also be saved and loaded to jump start training on the full data. 
<li>

### Download and Unzip Data

In [2]:
def bar_custom(current, total, width=80):
    print("Downloading: %d%% [%d / %d] bytes" % (current / total * 100, current, total))
import wget
import zipfile
zip_name = "train.zip"

url = "https://jrssbcrsefilesnait.blob.core.windows.net/3950data1/vegetable_image_dataset.zip"

if not os.path.exists(zip_name):
    wget.download(url, zip_name, bar=bar_custom)

with zipfile.ZipFile(zip_name, 'r') as zip_ref:
    zip_ref.extractall()

In [21]:
# Generate Datasets - you can change this if desired
# ENSURE FILE PATHS MATCH CORRECTLY
IMAGE_SIZE=(224,224)
train_dir='Vegetable Images/train'
val_dir='Vegetable Images/validation'
batch_size = 16
BUFFER = 150

# Load training data
train_ds = image_dataset_from_directory(
    train_dir,
    label_mode='categorical',
    image_size = IMAGE_SIZE,
    batch_size=batch_size
)

val_ds = image_dataset_from_directory(
    val_dir,
    label_mode='categorical',
    image_size = IMAGE_SIZE,
    batch_size = batch_size,
)

Found 15000 files belonging to 15 classes.
Found 3000 files belonging to 15 classes.


## Data Preparation

In [22]:
MONITOR = "val_accuracy"
BASE_EPOCHS = 2
BASE_PATIENCE = 5
MIN_DELTA = .02
MONITOR = "val_accuracy"
LOGS = "logs"
DIR_OUT = "kt_out"
PROJECT = "kt_basics"
FACTOR = 3
VALIDATION = .2

In [23]:
rescale = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255)
])

In [24]:
TPAIN = tf.data.AUTOTUNE

train_ds = train_ds.map(lambda x, y: (rescale(x), y))
train_ds = train_ds.shuffle(buffer_size=BUFFER)
train_ds = train_ds.prefetch(buffer_size=TPAIN)

val_ds = val_ds.map(lambda x, y: (rescale(x), y))
val_ds = val_ds.prefetch(buffer_size=TPAIN)

In [25]:
train_ds

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 15), dtype=tf.float32, name=None))>

## Custom Model Training

In [26]:
stopping = tf.keras.callbacks.EarlyStopping(monitor=MONITOR, patience=BASE_PATIENCE, mode="max", restore_best_weights=True)

In [27]:
def build_model(hp):
    model = Sequential()
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(layers.Conv2D(hp.Int("kernel_L"+str(i), min_value=32, max_value=512, step=16), (3, 3), input_shape=(224, 224, 3), padding="same"))
        model.add(layers.LeakyReLU())
        model.add(layers.MaxPooling2D())
    
    model.add(Flatten())
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(15, activation="sigmoid"))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [28]:
file_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tuner = kt.BayesianOptimization(build_model,
                        objective=MONITOR,
                        max_trials=5,
                        directory=DIR_OUT+"/"+file_time,
                        project_name=PROJECT,
                        overwrite=True)

In [29]:
# Get Results
tuner.search(train_ds, epochs=BASE_EPOCHS, validation_data=val_ds, callbacks=[stopping])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Build best model
best_model.build(input_shape=(None, 224,224,3))
best_model.summary()

Trial 2 Complete [00h 00m 12s]

Best val_accuracy So Far: None
Total elapsed time: 00h 00m 24s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
3                 |3                 |num_layers
192               |496               |kernel_L0
0.0001            |0.01              |learning_rate
256               |32                |kernel_L1
304               |32                |kernel_L2

Epoch 1/5
938/938 [==============================] - 146s 152ms/step - loss: 0.9088 - accuracy: 0.7095 - val_loss: 0.4707 - val_accuracy: 0.8573
Epoch 2/5
849/938 [==========================>...] - ETA: 12s - loss: 0.3013 - accuracy: 0.9079

KeyboardInterrupt: 

## Transfer Learning Model

## Test Best Models and Illustrate Results

In [ ]:
test_dir='Vegetable Images/test'
test_ds = image_dataset_from_directory(
    test_dir,
    label_mode='categorical',
    image_size = IMAGE_SIZE,
    batch_size = batch_size,
)

Found 3000 files belonging to 15 classes.
